<a href="https://colab.research.google.com/github/epsilon-deltta/sl-for-dl/blob/master/dl/test-lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### sonar 

In [1]:
from keras.models import Sequential
from keras.layers.core import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

import numpy
import pandas as pd
import tensorflow as tf

# seed 값 설정
seed = 0
numpy.random.seed(seed)
tf.random.set_seed(3)

df = pd.read_csv('../dataset/sonar.csv', header=None)

df.info()

# dataset = df.values
X = df.iloc[:,0:60]
Y_obj = df.iloc[:,60]

e = LabelEncoder()
e.fit(Y_obj)
Y = e.transform(Y_obj)
# 빈 accuracy 배열
accuracy = []
X = X.to_numpy()

In [1]:
from sklearn.model_selection import StratifiedKFold
# 10개의 파일로 쪼갬
n_fold = 2
skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)


# 모델의 설정, 컴파일, 실행
for train, test in skf.split(X, Y):
    model = Sequential()
    model.add(Dense(24, input_dim=60, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='mean_squared_error',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.fit(X[train], Y[train], epochs=100, batch_size=5)
    k_accuracy = "%.4f" % (model.evaluate(X[test], Y[test])[1])
    accuracy.append(k_accuracy)

model.save('sonar_model.h5')

from keras.models import Sequential, load_model

exsonar = load_model('sonar_model.h5')

### imdb  

In [51]:
# -*- coding: utf-8 -*-
# 코드 내부에 한글을 사용가능 하게 해주는 부분입니다.

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D
from keras.datasets import imdb

import numpy as np
import numpy
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.models import Model
# seed 값 설정
seed = 0
numpy.random.seed(seed)
tf.random.set_seed(3)
(x_train, ytr), (x_test, yte) = imdb.load_data(num_words=5000)
# x_train = x_train[:12500]
# x_test = x_test[:12500]
# ytr = ytr[:12500]
# yte = yte[:12500]
xtr = sequence.pad_sequences(x_train, maxlen=50)
xte = sequence.pad_sequences(x_test, maxlen=50)

In [94]:
# 모델의 설정
model = Sequential()
model.add(Embedding(5000, 100 )   )
model.add(Dropout(0.5))
model.add(Conv1D(64, 5, padding='valid', activation='relu',strides=1))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(55))
model.add(Dense(1))
model.add(Activation('sigmoid'))
# model.summary()

inputA = Input(shape=(50,) )
x = Embedding(5000,50)(inputA)
x = Dropout(0.5)(x)
x = Conv1D(64, 5, padding='valid', activation='relu',strides=1)(x)
x = MaxPooling1D(pool_size=4)(x)
x = LSTM(55)(x)
x = Dense(1,activation='sigmoid')(x)
model1 = Model(inputA,x)


import datetime , tensorflow as tf
%load_ext tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# 모델의 컴파일
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# 모델의 실행
history = model.fit(xtr, ytr, batch_size=250, epochs=1, validation_data=(xte, yte),callbacks=[tensorboard_callback])


In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
# 테스트 정확도 출력
print("\n Test Accuracy: %.4f" % (model.evaluate(x_test, y_test)[1]))

# 테스트 셋의 오차
y_vloss = history.history['val_loss']

# 학습셋의 오차
y_loss = history.history['loss']

# 그래프로 표현
x_len = numpy.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c="red", label='Testset_loss')
plt.plot(x_len, y_loss, marker='.', c="blue", label='Trainset_loss')

# 그래프에 그리드를 주고 레이블을 표시
plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()